## conda environments

When accessing this notebook via the JASMIN Jupyter Notebook service, select the correct conda environment from the list of available kernels.

## Imports

In [1]:
from os import path
import pandas as pd
import matplotlib.pyplot as plt
from data_classes import HealthData
from ONS_Excel import weekly_urls, weekly_sheet_names, weekly_row_indices

## Code

### Weekly mortality
Weekly data across England & Wales; by age, sex, region. Processed to look specifically at London region (all ages, all sexes).

In [ ]:
home_folder = path.join(path.abspath(""), "mortality_data")

#### Download & process the mortality data

Perks of using Excel files from the Office for National Statistics...

In [ ]:
urls = weekly_urls
sheet_names = weekly_sheet_names
row_indices = weekly_row_indices

In [ ]:
mortality_df = pd.DataFrame()

for year in range(2010, 2022):
    mortality_annual = HealthData(home_folder, url=urls[year])
    mortality_annual.download(verbose=False)
    df = mortality_annual.read_xls(sheet_names[year], verbose=False)
    rows = row_indices[year]
    annual_df = df.loc[rows]
    annual_df = annual_df.T.dropna().rename(columns={rows[0]:"date", rows[1]:"weekly_deaths"}).reset_index(drop=True).drop([0]).set_index("date")
    annual_df.index = pd.to_datetime(annual_df.index)
    if mortality_df.empty:
        mortality_df = annual_df.copy()
    else:
        mortality_df = mortality_df.append(annual_df.copy())
    print(f"{year} done...")
    
mortality_df.to_csv(path.join(home_folder, "weekly_mortality_London.csv"))

#### Load the processed data from .csv file

In [ ]:
mortality_annual = HealthData(home_folder, filename="weekly_mortality_London.csv")
mortality_df = mortality_annual.read_csv()
mortality_df.sample(5)

#### Plot the dataframe

In [ ]:
mortality_df.plot(legend=False, title="London").set_ylabel("weekly deaths")
plt.show()

### Daily mortality

In [2]:
home_folder = path.join(path.abspath(""), "mortality_data/daily")
# 1980 - 2014
url = "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/adhocs/005459dailydeathoccurrencesenglandregionsofenglandandwales1970to2014/dailydeathoccurrences19702014.zip"

In [3]:
mortality = HealthData(home_folder, url)

In [4]:
mortality.download()

Saved to dailydeathoccurrences19702014.zip
Contains zip files:
[0] Daily death occurrences, 1970-2014.xlsx


In [5]:
mortality.unzip("Daily death occurrences, 1970-2014.xlsx")

Unzipped Daily death occurrences, 1970-2014.xlsx.
Contains xlsx sheets: ['Notes and Terms & Conditions', 'National daily deaths', 'Regional daily deaths']


In [6]:
daily_1980_df = mortality.read_xls("Regional daily deaths")

Reading Daily death occurrences, 1970-2014.xlsx...


In [23]:
daily_1980_df = daily_1980_df.loc[daily_1980_df[3] == "E12000007", [0, 1, 2, 3, 4]]

In [24]:
daily_1980_df.columns = ["year", "month", "day", "region_code", "deaths"]

In [25]:
daily_1980_df["date"] = pd.to_datetime(daily_1980_df[["year", "month", "day"]])

In [34]:
daily_1980_df["deaths"]

13        245
23        214
33        256
43        235
53        260
         ... 
124143    171
124153    194
124163    210
124173    202
124183    200
Name: deaths, Length: 12418, dtype: object

In [39]:
daily_df = daily_1980_df[["date", "deaths"]].copy().set_index("date")

In [40]:
daily_df

,deaths
date,
1981-01-01,245
1981-01-02,214
1981-01-03,256
1981-01-04,235
1981-01-05,260
...,...
2014-12-27,171
2014-12-28,194
2014-12-29,210


### Population data

In [ ]:
home_folder = path.join(path.abspath(""), "population_data")

#### Download & process the population data

In [ ]:
url = "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland/mid2001tomid2019detailedtimeseries/regionalpopestimatesforenglandandwales19712019.xlsx"
pop_data = HealthData(home_folder, url=url)
pop_data.download()

In [ ]:
population_df = pd.DataFrame()

for sheet in ["Table 3", "Table 4"]:
    df = pop_data.read_xls(sheet, verbose=False)
    df = df.loc[df[0].apply(lambda x: isinstance(x, int))].reset_index(drop=True)[[0]].rename(columns={0:"date"}).join(df.loc[df[1]=="London"].reset_index(drop=True)[[2]].rename(columns={2:"total_population"})).set_index("date")
    df.index = pd.to_datetime(df.index, format="%Y") + pd.tseries.offsets.DateOffset(months=6) # Set these as mid-year estimates
    if population_df.empty:
        population_df = df.copy()
    else:
        population_df = population_df.append(df.copy())
        
population_df.to_csv(path.join(home_folder, "annual_population_London.csv"))

#### Load the processed data from .csv file

In [ ]:
pop_data = HealthData(home_folder, filename="annual_population_London.csv")
population_df = pop_data.read_csv(index_col="date")
population_df.sample(3)

#### Plot the dataframe

In [ ]:
population_df.plot(legend=False, title="London").set_ylabel("annual population")
plt.show()

#### Interpolation of annual data

In [ ]:
step_df = population_df.resample("D").asfreq().fillna(method="ffill")
linear_df = population_df.resample("D").asfreq().interpolate(method="linear")

In [ ]:
step_df.plot(legend=False, title="London annual population (mid-year step change)").set_ylabel("population")
linear_df.plot(legend=False, title="London daily population (interpolation)").set_ylabel("population")
plt.show()

### Calculating deaths as a fraction of population

In [ ]:
df = linear_df.join(mortality_df).dropna()
df["deaths_per_capita"] = df["weekly_deaths"]/df["total_population"]
df["deaths_per_capita"].plot(legend=False, title="London weekly deaths").set_ylabel("mortality per capita")
plt.show()

In [ ]:
df.to_csv(path.join(path.join(path.abspath(""), "mortality_data"), "weekly_mortality_percapita_London.csv"))